## Booze R Us Model

Fitting a model (or two) based on our proposal.

- **Goal:** Build a model to predict sales in a month for any given store.
- **Response Variable:** Monthly Sales
- **Possible Features:** store, month, county, population stuff, proximity stuff, alcohol categories

In [42]:
import duckdb as db 
con = db.connect()
import pandas as pd 
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [43]:
# MAIN TABLE
con.execute("""
        DROP TABLE IF EXISTS sales;
        CREATE TABLE sales AS 
        SELECT EXTRACT(MONTH FROM date) AS month, EXTRACT (YEAR FROM date) AS year,
            store, city, county, 
            category_name AS category, sale_bottles AS bottles, sale_dollars AS dollars
        FROM read_parquet('../data/iowa_liquor_2023_2025.parquet');
""")
sales = con.execute("SELECT * FROM sales").df()

# POPULATION
con.execute(
"""
        DROP TABLE IF EXISTS pop;
        CREATE TABLE population AS
        SELECT name AS county, year_1 AS year, popestimate AS population, over21, propOver21, median_age_tot AS median_age
        FROM read_csv_auto('../data/pop.csv');
"""
)
pop = con.execute("SELECT * FROM population").df()

# PROXIMITY
con.execute(
"""
        DROP TABLE IF EXISTS prox;
        CREATE TABLE proximity AS
        SELECT *
        FROM read_csv_auto('../data/proximity.csv');
"""
)
prox = con.execute("SELECT * FROM proximity").df()

In [44]:
sales.columns

Index(['month', 'year', 'store', 'city', 'county', 'category', 'bottles',
       'dollars'],
      dtype='object')

## Creating the Dataset

First, I am going to engineer the category column a little bit to use as features. Knowing which alcohol sells the best is could be useful for telling Booze R Us what they should buy in order to increase profits.

In [45]:
con.execute("""
    CREATE OR REPLACE TABLE sales AS
    SELECT *,
        CASE
            WHEN category ILIKE '%VODKA%' THEN 'Vodka'
            WHEN category ILIKE '%WHISK%' THEN 'Whiskey'
            WHEN category ILIKE '%TEQUILA%' OR category ILIKE '%MEZCAL%' THEN 'Tequila'
            WHEN category ILIKE '%RUM%' THEN 'Rum'
            ELSE 'Other'
        END AS super_category
    FROM sales
""")
sales = con.execute("SELECT * FROM sales").df()

In [46]:
sales.head()

,month,year,store,city,county,category,bottles,dollars,super_category
0,1,2023,4829,DES MOINES,POLK,100% AGAVE TEQUILA,12,261.00,Tequila
1,1,2023,4829,DES MOINES,POLK,AMERICAN VODKAS,60,418.80,Vodka
2,1,2023,4829,DES MOINES,POLK,IMPORTED FLAVORED VODKA,24,358.56,Vodka
3,1,2023,4829,DES MOINES,POLK,CREAM LIQUEURS,12,306.00,Other
4,1,2023,4829,DES MOINES,POLK,SPICED RUM,60,1124.40,Rum


Now I need to agreggate to create our appropriate observational units: monthly sales per store.

- Dollars (our response variable) will be summed. 
- Category will be made into new columns representing the distribution of category sales
    - e.g. 70% tequila, 20% vodkas, etc.
    - we will not use total bottles because this would be almost perfectly collinear 
    - answer questions like: 'what liquor should we sell more/less of?'

In [47]:
monthly_sales = con.execute(
""" 
    WITH month_totals AS (
        SELECT year, month, store, city, county,
            SUM(dollars) AS revenue
        FROM sales
        GROUP BY year, month, store, city, county
    ), category_totals AS (
        SELECT year, month, store, city, county,
            super_category,
            SUM(dollars) AS category_sales
        FROM sales
        GROUP BY year, month, store, city, county, super_category
    )
    SELECT mt.year, mt.month, mt.store, mt.city, mt.county,
        ROUND((SUM(CASE WHEN ct.super_category = 'Vodka' THEN ct.category_sales ELSE 0 END) / mt.revenue),2) AS vodka_ptc,
        ROUND((SUM(CASE WHEN ct.super_category = 'Whiskey' THEN ct.category_sales ELSE 0 END) / mt.revenue),2) AS whiskey_ptc,
        ROUND((SUM(CASE WHEN ct.super_category = 'Tequila' THEN ct.category_sales ELSE 0 END) / mt.revenue),2) AS tequila_ptc,
        ROUND((SUM(CASE WHEN ct.super_category = 'Rum' THEN ct.category_sales ELSE 0 END) / mt.revenue),2) AS rum_ptc,
        ROUND((SUM(CASE WHEN ct.super_category = 'Other' THEN ct.category_sales ELSE 0 END) / mt.revenue),2) AS other_ptc,
        mt.revenue
    FROM month_totals mt
    LEFT JOIN category_totals ct
        ON mt.year = ct.year AND mt.month = ct.month 
            AND mt.city = ct.city AND mt.county = ct.county
            AND mt.store = ct.store 
    GROUP BY mt.year, mt.month, mt.store, mt.city, mt.county, mt.revenue
    
"""
).fetchdf()

In [48]:
monthly_sales.head(2)

,year,month,store,city,county,vodka_ptc,whiskey_ptc,tequila_ptc,rum_ptc,other_ptc,revenue
0,2023,3,4873,GRANGER,DALLAS,0.20,0.58,0.05,0.12,0.05,6044.36
1,2023,3,4678,ADEL,DALLAS,0.22,0.44,0.02,0.18,0.14,44737.30


Now I will join with our other datasets, proximity and population. Using an inner join because it still leaves plenty of complete data for modelling. 

In [59]:
df = con.execute(
    """
        SELECT sales.*, 
            pop.population, pop.over21, pop.propOver21, pop.median_age,
            prox."# of stores within 5 mile radius" AS stores_within_5_miles,
            prox."Nearest other store (mi)" AS nearest_store_miles
        FROM monthly_sales sales
        JOIN pop
            ON LOWER(sales.county) = LOWER(pop.county) AND sales.year = pop.year
        JOIN prox
            ON sales.store = prox.store
        
    """
    ).fetchdf()

In [63]:
df.head(2)

,year,month,store,city,county,vodka_ptc,whiskey_ptc,tequila_ptc,rum_ptc,other_ptc,revenue,population,over21,propOver21,median_age,stores_within_5_miles,nearest_store_miles
0,2023,3,4873,GRANGER,DALLAS,0.20,0.58,0.05,0.12,0.05,6044.36,111593,82193,0.736543,36.3,1,4.785958
1,2023,3,4678,ADEL,DALLAS,0.22,0.44,0.02,0.18,0.14,44737.30,111593,82193,0.736543,36.3,6,0.264508


Now encode month to use as a categorical feature:

In [72]:
months = pd.get_dummies(df.month, prefix='month', drop_first=True)
df = pd.concat([df.drop(columns='month'), months], axis=1)
df.head()

,year,store,city,county,vodka_ptc,whiskey_ptc,tequila_ptc,rum_ptc,other_ptc,revenue,population,over21,propOver21,median_age,stores_within_5_miles,nearest_store_miles,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,2023,4873,GRANGER,DALLAS,0.20,0.58,0.05,0.12,0.05,6044.36,111593,82193,0.736543,36.3,1,4.785958,False,True,False,False,False,False,False,False,False,False,False
1,2023,4678,ADEL,DALLAS,0.22,0.44,0.02,0.18,0.14,44737.30,111593,82193,0.736543,36.3,6,0.264508,False,True,False,False,False,False,False,False,False,False,False
2,2023,4069,OTTUMWA,WAPELLO,0.19,0.21,0.03,0.48,0.09,44277.39,35417,27021,0.762939,39.2,21,0.154338,False,True,False,False,False,False,False,False,False,False,False
3,2023,3738,DUBUQUE,DUBUQUE,0.46,0.38,0.01,0.11,0.04,6166.14,98988,76855,0.776407,39.4,51,0.060371,False,False,True,False,False,False,False,False,False,False,False
4,2023,5226,DAVENPORT,SCOTT,0.25,0.18,0.16,0.02,0.39,16244.32,174589,134569,0.770776,39.6,73,0.115651,False,False,True,False,False,False,False,False,False,False,False


In [156]:
df = df.copy()
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df = df[df['revenue'] > 0]   
df.to_csv('../data/brs_model_data.csv', index=False)

## Linear Regression (from scratch)

First, the function to fit the model. It will take observed X and Y matrices and return the predictions and estimators.

In [196]:
def fit_lr(X, Y):
    
    X = np.column_stack([np.ones((X.shape[0], 1)), X.astype(float)]) # add column of ones
    B = np.linalg.pinv(X.T @ X) @ X.T @ Y

    return B

In [197]:
def predict_lr(X, B):
    
    X = np.column_stack([np.ones((X.shape[0], 1)), X.astype(float)]) # add column of ones
    Y_hat = X @ B

    return Y_hat

A cross validation function:

In [200]:
def lr_cross_val(data, features, response, k=5):

    # make k folds
    n = data.shape[0]
    indices = np.arange(n)
    np.random.shuffle(indices)
    fold_size = n // k
    folds = [indices[i*fold_size:(i+1)*fold_size] for i in range(k-1)]
    folds.append(indices[(k-1)*fold_size:])  # last fold gets the remainder

    # loop over folds and store their predictions
    predictions = np.zeros(n)
    for fold_idx in folds:
        # training indices = everything not in this fold
        train_idx = np.setdiff1d(indices, fold_idx)
        test_idx = fold_idx

        train = data.iloc[train_idx]
        test = data.iloc[test_idx]

        # fit model
        B = fit_lr(train[features].values, train[response].values)

        # predict on test set
        Y_hat = predict_lr(test[features].values, B)

        predictions[test_idx] = Y_hat

    return predictions


Fit the model:

- *NOTE:* I first fit with all possible features, then after assessing model fit, feature importance, and multicollinearity (all below), I came back up and changed the features. So this is just the final model here, not all iterations.

In [208]:
features = ['month_5','month_6','month_12',
        'vodka_ptc', 'whiskey_ptc', 'rum_ptc',
        'propOver21', 
        'stores_within_5_miles']
df['log_revenue'] = np.log(df['revenue'])
response = 'log_revenue'

# fit
Yhat = lr_cross_val(df, features, response, 5)

 Assess model fit:

In [217]:
Y = df[response].values
X = df[features].values
RSS = np.sum((Y - Yhat)**2)
TSS = np.sum((Y - np.mean(Y))**2)
R2 = 1 - RSS/TSS
n, p = X.shape  # after adding intercept
adj_R2 = 1 - (1 - R2) * (n - 1) / (n - p)
MAE = np.mean(np.abs(Y - Yhat))
RMSE = np.sqrt(RSS / n)

print(f"R2 = {R2:.4f}")
print(f"Adjusted R2 = {adj_R2:.4f}")
print(f"MAE = {MAE:.2f}")
print(f"RMSE = {RMSE:.2f}")

R2 = 0.1722
Adjusted R2 = 0.1720
MAE = 0.77
RMSE = 1.03


Fit the finalized model now on all the data and look at the estimators:

In [222]:
# fit a final model on all the data
B_final = fit_lr(X.values, Y)

In [223]:
coef_names = ['intercept'] + list(X.columns)
nonstd_df = pd.DataFrame({
    'feature': coef_names,
    'beta': B_final
})
nonstd_df = nonstd_df[nonstd_df['feature'] != 'intercept']
nonstd_df = nonstd_df.reindex(
    nonstd_df['beta'].abs().sort_values(ascending=False).index
)
print(nonstd_df)

                 feature      beta
4              vodka_ptc -2.737025
5            whiskey_ptc -2.674087
7             propOver21  0.997155
6                rum_ptc  0.562470
1                month_5  0.099256
2                month_6  0.078483
3               month_12  0.056263
8  stores_within_5_miles  0.005847
